# Segment Anything Model for Building Extraction From High-Resolution Satellite Imagery

This notebook shows how to extract buildings within refugee/IDP settlements (or buildings in general) from high-resolution satellite imagery using the Segment Anything Model (SAM) Adapter.<br>

The codes are adapted based on [SAM Adapter](https://github.com/tianrun-chen/SAM-Adapter-PyTorch) for training and [segment-geospatial](https://github.com/opengeos/segment-geospatial) for creating prediceted masks in the format of GeoTIFF and polygons in the format of ShapeFile.<br>

If you use Google Colab, make sure you use GPU runtime for this notebook. Go to `Runtime` -> `Change runtime type` and select `GPU` as the hardware accelerator.For training, it is better to use A100 GPU for the sake of memory and efficiency. <br>

These codes can be easily adapted for binary semantic segmentation applications in remote sensing. Feel free to use it for your own applications and implement in your local machine.<br>

In [1]:
import sagemaker
import boto3

sage_boto3 = boto3.client("sagemaker")
session = sagemaker.Session()
region = session.boto_session.region_name
bucket = "ubicubesambucket"
print(bucket)

ubicubesambucket


In [2]:
# send data to S3. SageMaker will take data from S3 in the future.
sk_prefix = "temp"
filename = 'requirements.txt'
filepath = session.upload_data(path=filename, bucket=bucket,key_prefix=sk_prefix)
print(filepath)

s3://ubicubesambucket/temp/requirements.txt


In [ ]:
file = open(filepath,'r')
file

In [ ]:
import boto3
import sagemaker

# Set up SageMaker session and bucket
session = sagemaker.Session()
bucket = 'ubicubesambucket'  # Replace with your S3 bucket name
sk_prefix = 'temp'  # Prefix for the file in S3
filename = 'requirements.txt'

# Upload file
filepath = session.upload_data(path=filename, bucket=bucket, key_prefix=sk_prefix)
print(f"File uploaded to: {filepath}")


In [1]:
import boto3

# Parse the file path to extract bucket and key
s3_path = filepath.replace("s3://", "")
bucket_name, key = s3_path.split('/', 1)

# Read the file using boto3
s3 = boto3.client('s3')

response = s3.get_object(Bucket=bucket_name, Key=key)
file_content = response['Body'].read().decode('utf-8')

print("File content:")
print(file_content)


NameError: name 'filepath' is not defined

In [13]:
# delete files from s3 bucket
bucket_name = "ubicubesambucket"
file_name = "temp/requirements.txt"

s3_client = boto3.client("s3")

response = s3_client.delete_object(Bucket=bucket_name, Key=file_name)
print(response)

{'ResponseMetadata': {'RequestId': '56P96T17V3WA2G82', 'HostId': 'ZF84/vVaAFQN89HwmT8MohIawmDPKmqvAdiWLDDnaKuNVvawYEZbIT/zXSEtd8n9f/5X0lBVqgHmSvccVBmWK8PmCK4kO+8zP8DFO7fV80E=', 'HTTPStatusCode': 204, 'HTTPHeaders': {'x-amz-id-2': 'ZF84/vVaAFQN89HwmT8MohIawmDPKmqvAdiWLDDnaKuNVvawYEZbIT/zXSEtd8n9f/5X0lBVqgHmSvccVBmWK8PmCK4kO+8zP8DFO7fV80E=', 'x-amz-request-id': '56P96T17V3WA2G82', 'date': 'Sat, 04 Jan 2025 17:10:59 GMT', 'server': 'AmazonS3'}, 'RetryAttempts': 0}}


In [11]:
from pprint import pprint
pprint(response)

{'ResponseMetadata': {'HTTPHeaders': {'date': 'Mon, 23 Dec 2024 19:33:38 GMT',
                                      'server': 'AmazonS3',
                                      'x-amz-id-2': 'gyFjC1hmNaFtYzIdzIcKjYQzFI2QFdD5oXd7SR1idjlDdp+Sw9JCF54MWDEkrgnoCy3hA9V2be4=',
                                      'x-amz-request-id': '3AMNF0T2GR5WVW9V'},
                      'HTTPStatusCode': 204,
                      'HostId': 'gyFjC1hmNaFtYzIdzIcKjYQzFI2QFdD5oXd7SR1idjlDdp+Sw9JCF54MWDEkrgnoCy3hA9V2be4=',
                      'RequestId': '3AMNF0T2GR5WVW9V',
                      'RetryAttempts': 0}}


In [1]:
import os
# import pathlib

# set up working directory and data folder
# path_base = "/content/drive/MyDrive/PhD_Research/SAM/SAM_Adapter_Final_4_Codes"
# os.chdir(path_base)
path = os.getcwd() # your current working directory where your codes are stored.
print(path)

import torch
print(torch.__version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

C:\YunyaGao\Ubicube\AWS\SAM\SAM_Adapter
2.0.1+cu118
Using device: cuda


## For Fine-tuning

#### Train and Inference

Avaialable input prompts: <br>
- parser = argparse.ArgumentParser()
- parser.add_argument('--config', default="configs/config_sam_vit_h.yaml", help="use the hyperparameters provided by SAM-Adapter")
- parser.add_argument('--data', default=None, help="different datasets")
- parser.add_argument('--upsample', default="1024", help="1024 or upscaled") 
- parser.add_argument('--size', default="small", help="small or large") 
- parser.add_argument('--uptype', default="", help="cubic or SR") 
- parser.add_argument('--epoch', default=10, help="epochs for training") 
- parser.add_argument('--model_save_epoch', default=2, help="the interval of saving trained models, do not save models in default due to big size of model.") 
- parser.add_argument('--inference_save_epoch', default=1, help="the interval of saving trained models") 
- parser.add_argument('--thres', default=0.5, help="the threshold to determine the binary map") 

`Change "path_data" in /run_sam/train.py & inference_noft.py & evaluation.py`


In [3]:
# command example1
!torchrun run_sam/train.py --data Dagaha2017 --epoch 3 --model_save_epoch 1 --inference_save_epoch 1 --size small --upsample 1024

NOTE: Redirects are currently not supported in Windows or MacOs.
[W ..\torch\csrc\distributed\c10d\socket.cpp:601] [c10d] The client socket has failed to connect to [Yunya]:29500 (system error: 10049 - 在其上下文中，该请求的地址无效。).
[W ..\torch\csrc\distributed\c10d\socket.cpp:601] [c10d] The client socket has failed to connect to [Yunya]:29500 (system error: 10049 - 在其上下文中，该请求的地址无效。).
C:\Users\gaoyu\anaconda3\envs\samUbi\lib\site-packages\mmcv\__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
[W ..\torch\csrc\distributed\c10d\socket.cpp:601] [c10d] The client socket has failed to connect to [Yunya]:29500 (system error: 10049 - 在其上下文中，该请求的地址无效。).
[W ..\torch\csrc\distributed\

In [3]:
# command example2
!torchrun run_sam/train.py --data Dagaha2017 --epoch 3 --model_save_epoch 1 --size small --upsample upscaled --uptype cubic

/home/yunya/anaconda3/envs/process_data_sam/lib/python3.10/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
ipynb_checkpoints folder not found.
config saved.
train dataset: size=11
  inp: shape=(3, 1024, 1024)
  gt: shape=(1, 1024, 1024)
model: #params=641.3M
model_grad_params:4245556 
model_total_params:641271604
Predicted probability map.                                                      
Save predicted probability map, binary map with threshold at 0.5 and shapefile.
Predicted probability map.
Save predicted probability map, binary map with threshold at 0.5 and shapefile.
Predicted probability map.                                          

## Inference based on pretrained models

In [1]:
!torchrun run_sam/inference_ft.py --data Dagaha2017 --model_save_epoch 1 --size small --upsample upscaled --uptype cubic

/home/yunya/anaconda3/envs/process_data_sam/lib/python3.10/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
config saved.
Predicted probability map.
Save predicted probability map, binary map with threshold at 0.5 and shapefile.
Predicted probability map.
Save predicted probability map, binary map with threshold at 0.5 and shapefile.


## Evaluation

In [2]:
from evaluation import evaluation_single_main

# select a specific prediction result and ground truth data to be evaluated
# this part can be furthur automated based on needs
path_gt = "/home/yunya/anaconda3/envs/Data/Dagaha2017/SAM/upscaled/test/cubic/gt/dagahaley2.tif"
path_pred = "/home/yunya/anaconda3/envs/SAM_Adapter/outputs/Dagaha2017/small/upscaled/cubic/epoch2/area2/pred_mask_bin0.5.tif"

evaluation_result = evaluation_single_main(path_pred, path_gt)

recall: 0.0
precision: 0.0
f1: 0.0
iou: 0.0
